<div align="center"><a href="https://www.nvidia.com/en-us/deep-learning-ai/education/"><img src="./images/DLI_Header.png"></a></div>

# Introduction to `NVIDIA Merlin dataloaders`

Before we begin building our neural networks, our [Amazon dataset](https://nijianmo.github.io/amazon/index.html) has changed slightly from the first lab. To read it into TensorFlow efficiently, we'll take advantage of a framework called [NVIDIA Merlin](https://developer.nvidia.com/nvidia-merlin).<br><br>

NVIDIA Merlin is a open source framework to accelerate recommendation systems end-to-end with GPUs: From ETL to training to deployment. Merlin has different components for each step. In this tutorial, we use [NVTabular](https://github.com/NVIDIA/NVTabular/)'s dataloader for TensorFlow, called `KerasSequenceLoader`. The dataloader can be the bottleneck of a training pipeline, which results into slow training. Recommender System problems often have a tabular data structure, so NVIDIA developed a custom-built, GPU-accelerated dataloader, which reads and prepares batches of data on the GPU.

You can read more in the official [documentation](https://nvidia-merlin.github.io/NVTabular/main/training/index.html) or in this [blog post](https://medium.com/nvidia-merlin/training-deep-learning-based-recommender-systems-9x-faster-with-tensorflow-cc5a2572ea49).

## Objective
This notebook demonstrates:
* How to use a [NVTabular dataloader](https://nvidia-merlin.github.io/NVTabular/main/training/tensorflow.html)
  * [1. Exploring the Data](#1.-Exploring-the-Data)
  * [2. Using a NVTabular dataloader](#2.-Using-a-NVTabular-dataloader)
* How to load data into a [TensorFlow Keras Functional Model](https://www.tensorflow.org/guide/keras/functional)
  * [3. Feeding the Data into a Model](#3.-Feeding-the-Data-into-a-Model)
  * [4. Wrap Up](#4.-Wrap-Up)

## 1. Exploring the Data
Let's load in our new dataset. It's very similar to what we used in Lab 1, but we've added some extra information. 
* `category_0_2_index` and `category_0_1_index`: We now have a numerical index for a couple category columns thanks to the strategies we learned with Content-based Filters. 
* `user_index`, `item_index`, and `brand_index`: A numerical index for each unique value thanks to [factorize](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.factorize.html).
* `valid`: True if validation/test row. False if training row.
* `als_prediction`: Our ALS prediction trained on the training rows described above.
* `user_embed_0`, `user_embed_1`, `item_embed_0`, `item_embed_1`: Our embeddings learned through ALS.

Let's load the data in and take a look with [cuDF](https://github.com/rapidsai/cudf).

TensorFlow will reserve the full available GPU memory. As the `KerasSequenceLoader` is a GPU dataloader, we need to limit the GPU memory TensorFlow will reserve.

In [ ]:
import os

os.environ["TF_MEMORY_ALLOCATION"] = "0.7"

In [ ]:
import cudf
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, concatenate, Dense

tf.__version__

In [ ]:
ratings = cudf.read_csv("data/task_2.csv")
ratings.head()

We should keep in mind which columns are categorical, and which are numerical.
* Example `CATEGORICAL_COLUMNS`: user_index, item_index, brand_index
* Example `NUMERICAL_COLUMNS`: price_filled, salesRank_Electronics

`KerasSequenceLoader` can read data from memory (`cuDF DataFrame`) or streams data from disk (`.csv` files, `.parquet` files or `.avro` files). We recommend using `.parquet` as it is a compressed file format and requires less data transfer from disk.

In [ ]:
tf.config.list_physical_devices('GPU')

## 2. Using a NVTabular dataloader

Let's import the `KerasSequenceLoader` and the required libraries.

In [ ]:
from nvtabular.loader.tensorflow import KerasSequenceLoader

import nvtabular as nvt

`KerasSequenceLoader` can load data from memory as `nvt.Dataset` or stream from disk. We will convert the cuDF dataframe to a `nvt.Dataset`.

In [ ]:
train_ds = KerasSequenceLoader(
    nvt.Dataset(ratings[~ratings["valid"]]),
    batch_size=1024,
    label_names=["overall"],
    cat_names=['user_index', 'item_index'],
    cont_names=[],
    shuffle=True,
    buffer_size=0.06
)

In [ ]:
valid_ds = KerasSequenceLoader(
    nvt.Dataset(ratings[ratings["valid"]]),
    batch_size=1024,
    label_names=["overall"],
    cat_names=['user_index', 'item_index'],
    cont_names=[],
    shuffle=False,
    buffer_size=0.06
)

`KerasSequenceLoader` require multiple parameters:
* `paths_or_dataset` is either a list of files, a glob pattern or a nvt.Dataset, which defines the data to use.
* `batch_size` is the number of elements per pull of the dataset.
* `label_names` is a list of column names, which is the target.
* `cat_names` is a list of column names, which are categorical features.
* `cont_names` is a list of column names, which are continuous features.
* `shuffle` activate shuffle of the dataset
* `buffer_size` refers to the fraction of total GPU memory to occupy with a buffered chunk. This is especially usefule to prepare asynchronously the next batch 

It might help to imagine a dataset like a deck of cards. <img src="images/cards.png" height=75 width=75>
* `shuffle` takes X cards off the top of the deck and shuffles them
* `batch_size` is the number of cards dealt to the player

Now that we have the code to build a pipeline, let's use it. We will split our dataset into `train_ds` and `valid_ds` by using the Boolean value of the `valid` column in our `ratings` DataFrame.

We can turn these datasets into [iterators](https://www.w3schools.com/python/python_iterators.asp) so we can test what our neural networks will be trained on. The `next` method can be used to call a new batch each time. Try running the next two cells. The results of them should have the same shape, but different data. 

In [ ]:
ds_itr = iter(train_ds)
next(ds_itr)

In [ ]:
next(ds_itr)

## 3. Feeding the Data into a Model

There are two common ways to make models with Keras. The [Sequential API](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential) is useful for making simple models, but for more freedom in our model architectures, we can use the [Functional API](https://www.tensorflow.org/guide/keras/functional). Let's try to make a model using our `train_ds` and `valid_ds` datasets.

Unlike the Sequential API where the model is defined by a list of layer objects, the functional API assigns layers through variables which can then call other layers as a function. Let's define the inputs to our model with [tf.keras.layers.Input](https://www.tensorflow.org/api_docs/python/tf/keras/Input).

In [ ]:
input_user_id = Input(name="user_index", dtype="int64", shape=(1,))
input_item_id = Input(name="item_index", dtype="int64", shape=(1,))

Right now, our model has an integer value for each index, but even though it's a number, it's actually more like categorical data. It would be useful if we could [one-hot encode](https://en.wikipedia.org/wiki/One-hot) our indexes. With that, each unique index would get a column with a `1` in the index corresponding to that row and `0` everywhere else.

For example, we could one-hot encode day of the week like this:

<img src="images/one-hot.png" width="500">

The trouble is, we have many thousands of users and products, which would result in a large number of parameters for our model. We'll do the same strategy with ALS: we'll make an [embedding](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding).

Embeddings in neural networks work a little differently than they do for ALS, but the goal is the same. We're trying to squeeze lots of data into a smaller space, but instead of learning this representation through ALS, we'll let the neural network learn them through backpropagation.

In [ ]:
embedding_size = 32
user_input_size = int(ratings["user_index"].max() + 1)
item_input_size = int(ratings["item_index"].max() + 1)
emb_user_id = Embedding(user_input_size, embedding_size)
emb_item_id = Embedding(item_input_size, embedding_size)

Next, we call the embedding layer with the related input tensor.

In [ ]:
out_emb_user_id = emb_user_id(input_user_id)
out_emb_item_id = emb_item_id(input_item_id)

If we take a look on the current output tensor, we can see the output shape.

shape=(None, 1, 32)

The first dimension is the batch_size. Keras uses `None` to mean any size, so we can have variable batch sizes.
The second dimension is the output (`embedding size`).

In [ ]:
out_emb_user_id

In [ ]:
out_emb_item_id

We have embedded the two categorical columns, `item_index` and `user_index`. So far, we have two independent graphs, which we have to combine. There are a lot of different "combination" types for Wide & Deep architectures types. We want to simply concatenate them along the embedding output dimension.

The output dimensions after concatenation should be (shape, 1, 64) ((shape, 1, 2*32)).

We can use the function [tf.keras.layers.concatenate](https://www.tensorflow.org/api_docs/python/tf/keras/layers/concatenate).

In [ ]:
x = concatenate([out_emb_user_id, out_emb_item_id], axis=1)

Next, we will define the hidden layers of our deep neural network. Let's add two hidden layers with 128 units and activation layer ReLU with [tf.keras.layers.Dense](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense).

In [ ]:
x = Dense(128, activation="relu", name="hidden_1")(x)
x = Dense(128, activation="relu", name="hidden_2")(x)

Our final layer is dense layer with only one hidden unit - our prediction. As we predict the user rating of a product, our prediction is between 1 and 5. Therefore, we do **NOT** set an activation function since the default activation is linear.

In [ ]:
output_tensor = tf.keras.layers.Dense(1, activation=None, name="pred")(x)

We will define the input tensors as a dictionary and initialize the [full model](https://www.tensorflow.org/api_docs/python/tf/keras/Model).

In [ ]:
input_tensor = {"user_index": input_user_id, "item_index": input_item_id}
model = tf.keras.Model(input_tensor, output_tensor)

To verify the structure of our model, we can visualize it as a graph.

In [ ]:
tf.keras.utils.plot_model(model, rankdir="LR")

We define the optimizer (Adam) and loss (mean squared error). We'll also add RMSE as a metric to compare to our Lab 1 Results.

In [ ]:
model.compile(
    optimizer="adam",
    loss="mse",
    metrics=[tf.keras.metrics.RootMeanSquaredError(name="rmse")],
)

All the pieces are together. It's time to train our model! Run the cell below to kick off training.

TensorFlow will automatically detect our GPU, but if we want to manually specify it, we can with [tf.device](https://www.tensorflow.org/api_docs/python/tf/device).

One small difference is, that `KerasSequenceLoader` requires to add the validation data as a callback with `KerasSequenceValidater`.

In [ ]:
from nvtabular.loader.tensorflow import KerasSequenceValidater

with tf.device("/gpu:0"):
    history = model.fit(train_ds, callbacks=[KerasSequenceValidater(valid_ds)], epochs=2)

## 4. Wrap Up

Hmm, we have a validation RMSE of about `1.15`. That's already better than our ALS result of about `1.20`! Can we do better? We sure can! Run the code below to shut down the kernel before moving on to the next notebook.

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<div align="center"><a href="https://www.nvidia.com/en-us/deep-learning-ai/education/"><img src="./images/DLI_Header.png"></a></div>